# 1. Import libraries

In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)

#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.style.use('ggplot')

import random
import scipy.sparse as sparse
import scipy.io

from keras.utils import to_categorical
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from skfeature.function.similarity_based import lap_score
from skfeature.utility import construct_W
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from skfeature.function.sparse_learning_based.UDFS import udfs
from skfeature.utility.sparse_learning import feature_ranking
import time

from concrete_autoencoder import ConcreteAutoencoderFeatureSelector
from keras.layers import Dense
import pandas as pd

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
#--------------------------------------------------------------------------------------------------------------------------------
def ETree(p_train_feature,p_train_label,p_test_feature,p_test_label,p_seed):
    clf = ExtraTreesClassifier(n_estimators=50, random_state=p_seed)
    
    # Training
    clf.fit(p_train_feature, p_train_label)
    
    # Training accuracy
    print('Training accuracy：',clf.score(p_train_feature, np.array(p_train_label)))
    print('Training accuracy：',accuracy_score(np.array(p_train_label),clf.predict(p_train_feature)))
    #print('Training accuracy：',np.sum(clf.predict(p_train_feature)==np.array(p_train_label))/p_train_label.shape[0])

    # Testing accuracy
    print('Testing accuracy：',clf.score(p_test_feature, np.array(p_test_label)))
    print('Testing accuracy：',accuracy_score(np.array(p_test_label),clf.predict(p_test_feature)))
    #print('Testing accuracy：',np.sum(clf.predict(p_test_feature)==np.array(p_test_label))/p_test_label.shape[0])

In [3]:
#--------------------------------------------------------------------------------------------------------------------------------
def write_to_csv(p_data,p_path):
    dataframe = pd.DataFrame(p_data)
    dataframe.to_csv(p_path, mode='a',header=False,index=False,sep=',')

# 2. Loading data

In [4]:
data_path="./Dataset/USPS.mat"
Data = scipy.io.loadmat(data_path)

data_arr=Data['X'].astype('float32')
label_arr=Data['Y'][:, 0]-1

C_train_x,C_test_x,C_train_y,C_test_y= train_test_split(data_arr,label_arr,test_size=0.2,random_state=seed)

print('Shape of C_train_x: ' + str(C_train_x.shape)) 
print('Shape of C_train_y: ' + str(C_train_y.shape)) 
print('Shape of C_test_x: ' + str(C_test_x.shape)) 
print('Shape of C_test_y: ' + str(C_test_y.shape)) 

Shape of C_train_x: (7438, 256)
Shape of C_train_y: (7438,)
Shape of C_test_x: (1860, 256)
Shape of C_test_y: (1860,)


In [5]:
key_feture_number=50

# 3. Model

In [6]:
def decoder(x):
    #x = Dense(key_feture_number)(x)
    x = Dense(data_arr.shape[1])(x)
    return x

In [7]:
C_train_x,C_test_x,C_train_y,C_test_y= train_test_split(data_arr,label_arr,test_size=0.2,random_state=seed)

print('Shape of C_train_x: ' + str(C_train_x.shape)) 
print('Shape of C_train_y: ' + str(C_train_y.shape)) 
print('Shape of C_test_x: ' + str(C_test_x.shape)) 
print('Shape of C_test_y: ' + str(C_test_y.shape)) 

Shape of C_train_x: (7438, 256)
Shape of C_train_y: (7438,)
Shape of C_test_x: (1860, 256)
Shape of C_test_y: (1860,)


In [8]:
start = time.clock()

selector = ConcreteAutoencoderFeatureSelector(K = key_feture_number, output_function = decoder, num_epochs = 1000)
selector.fit(C_train_x, C_train_x, C_test_x, C_test_x)

train_compressed_Data=data_arr[:, selector.get_support(indices=True)]

time_cost=time.clock() - start
write_to_csv(np.array([time_cost]),"./log/CAS_time"+str(key_feture_number)+".csv")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256)               0         
_________________________________________________________________
concrete_select (ConcreteSel (None, 50)                12801     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               13056     
Total params: 25,857
Trainable params: 25,856
Non-trainable params: 1
_________________________________________________________________
None
Train on 7438 samples, validate on 1860 samples
Epoch 1/1000
mean max of probabilities: 0.0046510096 - temperature 10.0
7438/7438 [==============================] - 1s 191us/step - loss: 0.2692 - val_loss: 0.2379
Epoch 2/1000
mean max of probabilities: 0.005155095 - temperature 9.942398
7438/7438 [==============================] - 1s 140us/step - loss: 0.2333 - val_loss: 0.228

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [9]:
train_compressed_Data.shape

(9298, 50)

# 4. Classifying

### Extra Trees

In [10]:
train_feature=C_train_x
train_label=C_train_y
test_feature=C_test_x
test_label=C_test_y

print('Shape of train_feature: ' + str(train_feature.shape)) 
print('Shape of train_label: ' + str(train_label.shape)) 
print('Shape of test_feature: ' + str(test_feature.shape)) 
print('Shape of test_label: ' + str(test_label.shape)) 

p_seed=seed
ETree(train_feature,train_label,test_feature,test_label,p_seed)

Shape of train_feature: (7438, 256)
Shape of train_label: (7438,)
Shape of test_feature: (1860, 256)
Shape of test_label: (1860,)
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.9666666666666667
Testing accuracy： 0.9666666666666667


In [11]:
C_train_selected_x,C_test_selected_x,C_train_y,C_test_y= train_test_split(train_compressed_Data,label_arr,test_size=0.2,random_state=seed)

print('Shape of C_train_x: ' + str(C_train_x.shape)) 
print('Shape of C_train_y: ' + str(C_train_y.shape)) 
print('Shape of C_test_x: ' + str(C_test_x.shape)) 
print('Shape of C_test_y: ' + str(C_test_y.shape)) 

Shape of C_train_x: (7438, 256)
Shape of C_train_y: (7438,)
Shape of C_test_x: (1860, 256)
Shape of C_test_y: (1860,)


In [12]:
train_feature=C_train_selected_x
train_label=C_train_y

test_feature=C_test_selected_x
test_label=C_test_y

print('Shape of train_feature: ' + str(train_feature.shape)) 
print('Shape of train_label: ' + str(train_label.shape)) 
print('Shape of test_feature: ' + str(test_feature.shape)) 
print('Shape of test_label: ' + str(test_label.shape)) 

p_seed=seed
ETree(train_feature,train_label,test_feature,test_label,p_seed)

Shape of train_feature: (7438, 50)
Shape of train_label: (7438,)
Shape of test_feature: (1860, 50)
Shape of test_label: (1860,)
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.9548387096774194
Testing accuracy： 0.9548387096774194


# 6. Reconstruction loss

In [13]:
from sklearn.linear_model import LinearRegression

def mse_check(train, test):
    LR = LinearRegression(n_jobs = -1)
    LR.fit(train[0], train[1])
    MSELR = ((LR.predict(test[0]) - test[1]) ** 2).mean()
    return MSELR

In [14]:
train_feature_tuple=(C_train_selected_x,C_train_x)
test_feature_tuple=(C_test_selected_x,C_test_x)

reconstruction_loss=mse_check(train_feature_tuple, test_feature_tuple)
print(reconstruction_loss)

0.009840189
